In [ ]:
#!/usr/bin/env python
# coding: utf-8

In[3]:

In [ ]:
pwd

In[4]:

In [ ]:
get_ipython().system('pip install keras==2.2.4')
get_ipython().system('pip install tensorflow==2.7.3')

Model Building

Import The Required Model Building Libraries<br>


In[6]:

mport imagedatagenerator

In [ ]:
from keras.preprocessing.image import ImageDataGenerator

In[7]:

raining datagen

In [ ]:
train_datagen=ImageDataGenerator(rescale=1./255,shear_range=0.2,zoom_range=0.2,horizontal_flip=True)

In[8]:

esting datagen

In [ ]:
test_datagen=ImageDataGenerator(rescale=1./255)

In[9]:

In [ ]:
import os, types
import pandas as pd
from botocore.client import Config
import ibm_boto3

In [ ]:
def __iter__(self): return 0

@hidden_cell<br>
The following code accesses a file in your IBM Cloud Object Storage. It includes your credentials.<br>
You might want to remove those credentials before you share the notebook.

In [ ]:
cos_client = ibm_boto3.client(service_name='s3',
    ibm_api_key_id='_1LlD6NcpasdaL30Xs39-l5jOQzH2d_e4VdaohKJpDI1',
    ibm_auth_endpoint="https://iam.cloud.ibm.com/oidc/token",
    config=Config(signature_version='oauth'),
    endpoint_url='https://s3.private.us.cloud-object-storage.appdomain.cloud')

In [ ]:
bucket = 'realtimecommunicationsystempowere-donotdelete-pr-55zbqfdoz8rooe'
object_key = 'conversation engine for deaf and dumb .zip'

In [ ]:
streaming_body_3 = cos_client.get_object(Bucket=bucket, Key=object_key)['Body']

Your data file was loaded into a botocore.response.StreamingBody object.<br>
Please read the documentation of ibm_boto3 and pandas to learn more about the possibilities to load the data.<br>
ibm_boto3 documentation: https://ibm.github.io/ibm-cos-sdk-python/<br>
pandas documentation: http://pandas.pydata.org/

In[10]:

In [ ]:
from io import BytesIO
import zipfile
unzip = zipfile.ZipFile(BytesIO(streaming_body_3.read()),'r')
file_paths = unzip.namelist()
for path in file_paths:
    unzip.extract(path)

In[11]:

In [ ]:
pwd

In[12]:

In [ ]:
import os
filenames = os.listdir('/home/wsuser/work/conversation engine for deaf and dumb')

IMPORTING tensorflow

In[13]:

In [ ]:
import tensorflow as tf
import os

**Initialize The Model**

In[14]:

reate model

In [ ]:
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Convolution2D
from keras.layers import MaxPooling2D
from keras.layers import Dropout
from keras.layers import Flatten 
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In[15]:

In [ ]:
import numpy as np
import matplotlib.pyplot as plt #to view graph in colab itself
import IPython.display as display
from PIL import Image
import pathlib

Unzipping the dataset

Applying ImageDataGenerator to training set

In[16]:

In [ ]:
x_train=train_datagen.flow_from_directory('/home/wsuser/work/conversation engine for deaf and dumb/Dataset/training_set',target_size=(64,64),batch_size=200,
                                          class_mode='categorical',color_mode="grayscale")

Applying ImageDataGenerator to test set

In[17]:

In [ ]:
x_test=test_datagen.flow_from_directory('/home/wsuser/work/conversation engine for deaf and dumb/Dataset/test_set',target_size=(64,64),batch_size=200,
                                          class_mode='categorical',color_mode="grayscale")

In[18]:

In [ ]:
a=len(x_train)
b=len(x_test)

Length of training set

In[19]:

In [ ]:
print(a)

Length of test set

In[20]:

In [ ]:
print(b)

**Add Layers**

In[22]:

reate model

In [ ]:
model=Sequential()

Add The Convolution Layer

In[23]:

In [ ]:
model.add(Convolution2D(32,(3,3),input_shape=(64,64,1),activation='relu'))

Add Pooling Layer

In[24]:

In [ ]:
model.add(MaxPooling2D(pool_size=(2,2)))

Add The Flatten Layer

In[25]:

In [ ]:
model.add(Flatten())

Adding The Dense Layers

In[26]:

st hidden layer

In [ ]:
model.add(Dense(units=512,activation='relu'))
#2nd hidden layer
model.add(Dense(units=261,activation='relu'))

In[27]:

utput layer

In [ ]:
model.add(Dense(units=9,activation='softmax'))

Compile The Model

In[28]:

In [ ]:
model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])

Fit The Model

In[29]:

In [ ]:
model.fit_generator(x_train,steps_per_epoch=len(x_train),epochs=10,validation_data=x_test,validation_steps=len(x_test))

Save The Model

In[30]:

In [ ]:
model.save('IBM_WATSON_STUDIO_TRAINED_MODEL.h5')

In[31]:

In [ ]:
get_ipython().system('tar -zcvf Sign-language-model.tgz IBM_WATSON_STUDIO_TRAINED_MODEL.h5')

In[34]:

In [ ]:
ls -1

In[35]:

In [ ]:
get_ipython().system('pip install watson-machine-learning-client --upgrade')

In[42]:

In [ ]:
from ibm_watson_machine_learning import APIClient
wml_credentials ={
    "url":"https://us-south.ml.cloud.ibm.com",
    "apikey":"65yy2Zf2RrLpAmFe2T29AAfcRuaKhTh4wLWfpuAHsP32"
}
client = APIClient(wml_credentials)

In[45]:

In [ ]:
client = APIClient(wml_credentials)

In[46]:

In [ ]:
def guid_from_space_name(client, space_name):
    space = client.spaces.get_details()
    return(next(item for item in space['resources'] if item['entity']['name'] == space_name)['metadata']['id'])
    

In[48]:

In [ ]:
space_uid = guid_from_space_name(client, 'signlanguage')
print("Space UID = " + space_uid)

In[49]:

In [ ]:
client.set.default_space(space_uid)

In[50]:

In [ ]:
client.software_specifications.list()

In[51]:

In [ ]:
software_spec_uid = client.software_specifications.get_uid_by_name("tensorflow_rt22.1-py3.9")
software_spec_uid

In[52]:

In [ ]:
model_details = client.repository.store_model(model='Sign-language-model.tgz', meta_props={
    client.repository.ModelMetaNames.NAME: "CNN",
    client.repository.ModelMetaNames.SOFTWARE_SPEC_UID: software_spec_uid,
    client.repository.ModelMetaNames.TYPE: "tensorflow_2.7"})
model_id = client.repository.get_model_uid(model_details)

Import The Packages And Load The Saved Model

In[38]:

In [ ]:
from tensorflow.keras.models import load_model
import numpy as np
import cv2
from tensorflow.keras.preprocessing import image

In[45]:

oad the model

In [ ]:
model=load_model('aslpng2.h5')

In[46]:

In [ ]:
img=image.load_img('/content/Dataset/test_set/A/10.png',target_size=(400,500))
img

Load The Test Image, Pre-Process It And Predict

In[79]:

In [ ]:
from skimage.transform import resize
def detect(frame):
  img=resize(frame,(64,64,1))
  img=np.expand_dims(img,axis=0)
  if(np.max(img)>1):
    prediction=model.predict(img)
    print(prediction)
    prediction=model.predict_classes(img)
    print(prediction)

In[73]:

In [ ]:
arr= image.img_to_array(img)

In[67]:

In [ ]:
frame=cv2.imread('/content/Dataset/test_set/A/10.png')
data=detect(frame)
from google.colab.patches import cv2_imshow
cv2_imshow(frame)
cv2.waitKey(0)
cv2.destroyAllWindows()